## DDQN in Hockey Enviroment - Evaluation

Evaluate a trained DDQN against various environments

Base DQN implementation adapted from HW7

In [13]:
from importlib import reload

import numpy as np
from tqdm.notebook import tqdm

import DDQN.DDQN as ddqn
from DDQN.DDQN import DDQNAgent
from DDQN.evaluation import compare_agents, display_stats
import hockey.hockey_env as h_env

reload(h_env)
reload(ddqn)

<module 'DDQN.DDQN' from '/home/kivanc/ders/RL/project/RL-Hockey/DDQN/DDQN.py'>

In [14]:
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

## Environment & Agent Initialization

In [15]:
env = h_env.HockeyEnv(mode=h_env.Mode.NORMAL)
env.reset()

(array([-3.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  3.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.7543292 ,  0.03788853,  0.        ,
         0.        ,  0.        ,  0.        ]),
 {'winner': 0,
  'reward_closeness_to_puck': 0.0,
  'reward_touch_puck': 0.0,
  'reward_puck_direction': 0.0})

In [16]:
STATE_PATH = "./DDQN/models/hockey_normal_dev/"

agent_player = DDQNAgent(
    env.observation_space,
    env.discrete_action_space
)
agent_player.load_state(STATE_PATH)

agent_opp_weak = h_env.BasicOpponent(weak=True)
agent_opp_strong = h_env.BasicOpponent(weak=False)

## Evaluation

### Winning Rates Against Opponents & Match Statistics

In [17]:
weak_opp_stats = compare_agents(
    agent_player, agent_opp_weak, env, num_matches=1000, tqdm=tqdm
)
strong_opp_stats = compare_agents(
    agent_player, agent_opp_strong, env, num_matches=1000, tqdm=tqdm
)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [18]:
print("Weak Opponent Stats:")
display_stats(weak_opp_stats)

print("Strong Opponent Stats:")
display_stats(strong_opp_stats)

Weak Opponent Stats:
Player Observation Mean:
  x pos player one: -1.3339319781093817
  y pos player one: -0.6117579154434678
  angle player one: -0.0904754886006979
  x vel player one: 0.8691245156979047
  y vel player one: -0.5099625184104828
  angular vel player one: -0.09411099712039225
  x player two: 2.5580066272558577
  y player two: 0.06302552246696394
  angle player two: -0.004924118873475803
  y vel player two: -0.06429875971180268
  y vel player two: -0.024798722600795024
  angular vel player two: -0.0018472111019365853
  x pos puck: -0.07213065247695523
  y pos puck: 0.11566362079153174
  x vel puck: -1.1673583811750676
  y vel puck: 0.08016062137701138
  left player puck keep time: 0.9419729747348228
  right player puck keep time: 0.9132900606252503

Relative Std. Change in Agent Observations:
  x pos player one: -0.14199693003479125
  y pos player one: 0.5632918792660615
  angle player one: 0.771831603701401
  x vel player one: -0.21203642040755127
  y vel player one: 0.6

### Rendered Demonstration Against Strong Opponent

In [ ]:
obs_buffer = []
reward_buffer = []
obs, _ = env.reset()
obs_opp = env.obs_agent_two()

done = False
trunc = False
step = 0
while not (done or trunc):
    step += 1
    env.render()

    a1_discr = agent_player.act(obs)
    a1 = env.discrete_to_continous_action(a1_discr)
    a2 = agent_opp_strong.act(obs_opp)

    obs, r, done, trunc, _ = env.step(np.hstack([a1, a2]))
    obs_buffer.append(obs)
    reward_buffer.append(r)

    obs_opp = env.obs_agent_two()

    if done or trunc:
        print(f"Episode done in {step} steps")
        break

In [ ]:
env.close()